#### first upscale, then combined then downscale to give importance one or the other?

In [1]:
from PIL import Image
import numpy as np
import warnings
import requests
from core import _from_url, _handle_images, _combine_images, _get_hidden_images

In [ ]:
url = "https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/dog-puppy-on-garden-royalty-free-image-1586966191.jpg?crop=1.00xw:0.669xh;0,0.190xh&resize=980:*"
images = [np.array(Image.open(f"dog{i}.jpg")) for i in range(1, 5)]
images.append("https://i.insider.com/5484d9d1eab8ea3017b17e29?width=600&format=jpeg&auto=webp")
images.append(url)
TEXT = "be at 36.896891-30.713323 at 11:00:00 GMT"
# since the data is an image, the program will use 8 bits (0, 255)
BYTE = 8

In [ ]:
ims = _handle_images(images)

In [ ]:
Image.fromarray(_combine_images(ims[:4]) >> 0 << 6)

In [ ]:
def _text_to_binary(text: str, max_length: int) -> np.ndarray:
    """
    """
    # chars to ascii
    ord_text = [ord(ch) for ch in text]
    if max(ord_text) > 255:
        warnings.warn("""
            Warning: Chars with ascii codes higher than 
                     255 is not supported and will be lost""")
        ord_text = [min(ord_, 255) for ord_ in ord_text]
    
    # padding the text with whitespace until it reaches the max length 
    ord_text = ord_text + [32 for _ in range(max_length-len(ord_text))]
    # converting ascii codes to their byte represantation
    ord_text = [("0"*BYTE+"{0:b}".format(num))[-BYTE:] for num in ord_text]
    # converting it to 1's and 0's
    return np.array(list(map(int, "".join(ord_text))))

one function that takes:
- multiple images -> combines them and returns the combined image
- one image and key:
    - if text is passed:
        - encodes the image and returns image with encoded data
    - if no text:
        - decodes the image and returns the text
- one image and num_hidden:
    - number of hidden images needs to be passed
    - return the extracted images.
- raise error. or adds some default and try to return something meaningful

In [ ]:
def hider(*images, key: int=None, num_hidden: int=None, text: str=None):
    if len(images) == 0:
        raise Exception("Error: No image passed.")
    images = _handle_images(images)
    if len(images) > 1:
        return _combine_images(images)
    # just to indicate
    assert len(images) == 1
    image = images[0]
    
    if not key and not num_hidden:
        raise Exception("Error: key or num_hidden is needed.")
    if key:
        if num_hidden:
            raise Exception("""Error: Passing both key and num_hidden is unnecessary. 
                                            What is your intention?""")
        if text:
            return _encode_image_with_text(image, text=text, key=key)
        else: # no text
            return _decode_image(image, key=key)
    # just to indicate
    assert num_hidden != None
    return _get_hidden_images(image, num_hidden)

In [ ]:
Image.fromarray(hider(IMAGES[0], IMAGES[1]))

In [ ]:
Image.fromarray(hider(IMAGES[0], key=1, text="asdasldksal"))

In [ ]:
def _encode_image_with_text(org_data: np.ndarray, 
            text: str, 
            key: int, 
            max_length: int=50, 
            nnoise: int=1) -> np.ndarray:
    """
    """
    # the function works with the flattened data
    data = org_data.reshape(np.prod(org_data.shape)).copy()
    # the key is used for generating the random indexes in the flattened data
    np.random.seed(key)
    if max_length*BYTE > data.shape[0]:
        raise Exception("Error: Image size is to small for the text.")
    idxs = np.random.choice(data.shape[0], max_length*BYTE, replace=False)
    # converting the text to 1's and 0's for storing
    hidden = _text_to_binary(text, max_length)
    # out of all the indexes, it only changes the ones that dont 
    # match already, by subtracting 1 (changes the leftmost bit)
    data[idxs[hidden != data[idxs]%2]] -= 1
    
    # if someone else has the original image, part of the text could be
    # uncovered, so now will add random noise(salt?) to some other indexes
    # nnoise is a variable that controls the number of noise is added
    # more nnoise is more secure but also means more changes to the original
    if nnoise*max_length*BYTE > data.shape[0]:
        nnoise = data.shape[0]//(max_length*BYTE)
        warnings.warn("Warning: Lowering the nnoise.")
    noise_idxs = np.random.choice(data.shape[0], nnoise*max_length*BYTE, replace=False)
    # filtering the ones thats in the hidden text's indexes
    noise_idxs = np.setdiff1d(noise_idxs, idxs)
    # adding the noise
    data[noise_idxs] -= 1
    # return the reshaped flattened data with the original shape
    return data.reshape(org_data.shape)

In [ ]:
def _decode_image(encoded_data: np.ndarray, 
            key: int=0, 
            max_length: int=50) -> str:
    # flattenes the encoded image
    data = encoded_data.reshape(np.prod(encoded_data.shape)).copy()
    # sets the key and gets the indexes of the hidden text
    np.random.seed(key)
    idxs = np.random.choice(data.shape[0], max_length*BYTE, replace=False)
    # takes the leftmost bits from the indexes
    s = "".join(map(str, data[idxs]%2))
    # splits the bits into groups of bytes
    s = [s[i:i+BYTE] for i in range(0, len(s), BYTE)]
    # first, converts each of the bytes to ascii code then to chars
    # and gets the encoded text within the image
    s = "".join([chr(int(bin_, base=2)) for bin_ in s])
    # removes paddings and returns the text
    return s.strip()